# Login and Initialize
## Login Using Spotify
After running this click Sign-In and login to Spotify. **Note** you cannot run the whole Notebook through. Run cell #1 first.

In [ ]:
%load_ext autoreload
%autoreload 2
from ipyauth import ParamsSpotify, Auth
auth = Auth(ParamsSpotify(
    redirect_uri='http://localhost:8888/callback', 
    client_id="9e4657eefbac41afa98c61f590d8fd51"))
auth

## Fetch the Library
Initialize the Spotify object with a `logger` and fetch the library DataFrame. Check it works by outputting the Profile image.

In [ ]:
import requests
from IPython.display import Image
from spotify import Spotify
from pandas import DataFrame as df
from datetime import datetime,timedelta,date
import json
import logging

logging.basicConfig()
defaultLogger = logging.getLogger()
defaultLogger.setLevel(logging.ERROR)
logger = logging.getLogger('SpotifyLab')
logger.setLevel(logging.DEBUG)

spot = Spotify(auth, logger = logger)
lib = spot.fetchLibraryDataFrame( cache=False)
#lib.columns.values
Image(url=spot.imageUrl, width=50)

# Playlist Updates

## A Running Playlist

This playlist has high tempo and energy. The preference is for instrumentals with high danceability. Avoid long songs to keep the tracks changing.

In [ ]:
newPlaylist = lib[(lib.energy>0.90) & (lib.tempo>100) & (lib.instrumentalness>0.5) & (lib.duration_ms<1000*60*6) & (lib.danceability>0.4) & (lib.loudness>-11)]
# Exclude some artists from this playlist
newPlaylist = newPlaylist[~newPlaylist.artist.isin(["Rodrigo y Gabriela","2CELLOS","Faithless","Doves"])]
spot.updatePlaylist("Auto Run Fast",newPlaylist)

## Unpopular or Hispter?
This playlists tracks tracks with low popularity. This is useful for checking if you added the most commonly played track (e.g. a version before it was remastered).

In [ ]:
spot.updatePlaylist("Are You Sure",lib[lib.track_popularity<0.10])

## Decades of Music
These playlists bucket the music in your library into decades using the release date. 

In [ ]:
spot.updatePlaylist("auto 1960s",lib[lib.released<datetime(1970,1,1)])
spot.updatePlaylist("auto 1970s",lib[(lib.released>datetime(1970,1,1)) & (lib.released<datetime(1980,1,1))])
spot.updatePlaylist("auto 1980s",lib[(lib.released>datetime(1980,1,1)) & (lib.released<datetime(1990,1,1))])
spot.updatePlaylist("auto 1990s",lib[(lib.released>datetime(1990,1,1)) & (lib.released<datetime(2000,1,1))])
spot.updatePlaylist("auto 2000s",lib[(lib.released>datetime(2000,1,1)) & (lib.released<datetime(2010,1,1))])
spot.updatePlaylist("auto 2010s",lib[(lib.released>datetime(2010,1,1)) & (lib.released<datetime(2020,1,1))])
spot.updatePlaylist("2019 So Far",lib[lib.released>datetime(2019,1,1)])


## Recently Added
The music you added in the last couple of months.

In [ ]:
spot.updatePlaylist("Auto Recent",lib[lib.added_at>(datetime.now()-timedelta(days=60)).isoformat()])
spot.updatePlaylist("New Mix Takes",lib[(lib.released>(datetime.now()-timedelta(days=90)))
                                        & (lib.energy>0.3)])

## Mood Music
Music for every occasion. Low energy dinner parties, going to sleep or concentrating at work.

In [ ]:
spot.updatePlaylist("Dinner Party",lib[(lib.acousticness>0.9) & (lib.energy<0.6)])
spot.updatePlaylist("Night Nights",lib[(lib.acousticness>0.9) & (lib.energy<0.3) & (lib.danceability<0.3)])
spot.updatePlaylist("To Work To",lib[(lib.instrumentalness>0.9) & (lib.energy>0.6) ])
spot.updatePlaylist("Quiet Work",lib[(lib.instrumentalness>0.9) & (lib.danceability<0.6) & (lib.energy>0.3) ])
# With new Music Only
spot.updatePlaylist("Recent Work Music",lib[
    (lib.instrumentalness>0.9) & 
    (lib.energy>0.6) & 
    (lib.added_at>(datetime.now()-timedelta(days=90)).isoformat() )])
spot.updatePlaylist("Recent Quiet Work",lib[
    (lib.instrumentalness>0.9) & 
    (lib.danceability<0.6) & 
    (lib.energy>0.3) &
    (lib.added_at>(datetime.now()-timedelta(days=90)).isoformat()) ])

## More Analysis
It's easy to add the same track twice, from different albums. This can cause duplicates in playlists. We can use Pandas `groupby` to scan for that

In [ ]:
# Duplicates in the library
ff = lib.groupby(["track_name","artist"]).count()
(ff[ff.track_uri>1]).index.values